### Import library

In [1]:
import os
os.chdir('../..')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [4]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_JLf6tBJgw8a3vWphnqkmNG3NVs2M

 


In [6]:
ace.check_session_timeout(s)

14393.752114

알파 믹싱을 위한 템플릿입니다.

## Alpha Mixing Templates

In [18]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

'\nsignal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));\n\nsignal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));\n\nsignal1 * (1 + signal2)\n'

In [19]:
'''
signal1 = oth176_lmt_close_eq_score ;
signal2 = oth176_malta_eq_score ;
k = ts_corr(signal1, signal2, 30) ;
k*regression_neut(signal1, signal2)

[아이디어]
경향성이 비슷한 알파 활용하여 corr로 weight를 부여
'''

'\nsignal1 = oth176_lmt_close_eq_score ;\nsignal2 = oth176_malta_eq_score ;\nk = ts_corr(signal1, signal2, 30) ;\nk*regression_neut(signal1, signal2)\n\n[아이디어]\n경향성이 비슷한 알파 활용하여 corr로 weight를 부여\n'

이곳에 활용할 만한 템플릿들을 적어두고, 후에 골라 쓰시면 보기 편하실겁니다!

## Read all passed alphas

Results 파일에 있는 여태까지 시뮬레이션 해 저장해 두었던 알파 stats.csv 파일들을 불러옵니다.

In [3]:
file_direc = glob.glob('Results/signal/*')
file_direc

['Results/signal\\A_signalform.csv',
 'Results/signal\\B_signalform.csv',
 'Results/signal\\C_signalform.csv',
 'Results/signal\\D_signalform.csv']

In [4]:
expressions = {}
expressions['A'] = pd.read_csv(file_direc[0])['expression']
expressions['B'] = pd.read_csv(file_direc[1])['expression']
expressions['C'] = pd.read_csv(file_direc[2])['expression']
expressions['D'] = pd.read_csv(file_direc[3])['expression']

이제 expressions 라는 dictionary 에 각각의 템플릿에서 선별된 알파들의 expression 이 들어있습니다.

이때 주의해야 할 점은, 알파들의 expression 의 형태를 조금 변형해 주어야 한다는 것입니다. 예시를 보겠습니다.

In [5]:
expressions['A'][0]

'pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_0 = group_zscore(ts_ir(returns, 252), industry);alpha_A_0 = regression_neut(stock_A_0, factor_A_0);signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5)'

In [ ]:
'''
pro_A_0 = oth460_1l_acda;
stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);
factor_A_0 = group_zscore(ts_ir(returns, 252), industry);
alpha_A_0 = regression_neut(stock_A_0, factor_A_0);
signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5)
'''

위 expression 에는 두 가지 특이사항이 있습니다. 우선 첫 째로, 마지막 코드라인이 시그널 형태로 끝나는 것이 아닌, 'signal_A_0' 라는 변수명에 시그널을 대입하는 식으로 끝이 납니다. 알파 믹싱을 위해서는 이렇게 각각의 expression 마지막 코드라인을 시그널이 아니라 변수명에 시그널을 대입하는 식으로 변경해 주어야 합니다. <br>
또, 모든 변수 명에 이 expression 의 템플릿 호칭과 index 가 표기되어 있습니다. 이 expression 은 A 템플릿의 첫 번째 알파이니 'A_0' 를 모든 변수명에 붙여둔 모습입니다. 이는 같은 변수명을 다시 호출하는 게 불가능한 Fast Expression 문법 때문에 그렇습니다. 만약 같은 템플릿에서 나온 두 expression 을 써서 믹싱을 한다면 문제가 생기겠죠? 이처럼 모든 변수명을 그 알파의 고유 이름으로 바꾸어야 에러가 생기지 않습니다.

## Template AB mixing

템플릿 A와 B의 알파들을 믹싱하는 예제를 보여 드리겠습니다.

In [59]:
#region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01

In [6]:
a = expressions['A'][:32]
b = expressions['B'][:24]

In [13]:
alpha_list_AB = []

for i in range(0, 5):
    for j in range(0, 5):
        alpha_list_AB.append(ace.generate_alpha(f'{a[i]};{b[j]};add(scale_down(signal_A_{i}), scale_down(signal_B_{j}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01, decay = 12))

alpha_list_AB

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 12,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'pro_A_25 = oth460_1l_fbds;stock_A_25 = ts_zscore(ts_backfill(anl69_cps_best_eeps_cur_yr, 21),21);factor_A_25 = group_zscore(ts_ir(returns, 252), industry);alpha_A_25 = vector_neut(stock_A_25, factor_A_25);signal_A_25 = trade_when(pro_A_25<=0.5, alpha_A_25, pro_A_25>0.5);entry_B_20 = ts_rank( ts_std_dev(oth460_clv_class, 21), 252 ); maintenance_B_20 = -ts_regression(rsk70_mfm2_gemtrd_srtindcnt, ts_delay(rsk70_mfm2_gemtrd_size, 1), 252); quit_B_20 = entry_B_20 < 0.5;signal_B_20 =  trade_when(entry_B_20>0.55, maintenance_B_20, quit_B_20);add(scale_down(signal_A_25), scale_down(signal_B_20), filter = false)'},
 {'type': 'R

위 식으로 템플릿 A와 B의 상위 5개의 알파들을 add, scale_down 함수를 이용해 믹싱할 수 있습니다. 믹싱을 하실 때 주의하셔야 할 점은, 사용하는 두 알파들의 region 과 universe 가 같아야 한다는 점입니다. 또, neutralization 변수값에 민감한 알파라면 이 또한 주의하셔야 합니다.

In [14]:
result = ace.simulate_alpha_list_multi(s, alpha_list_AB)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [14:59<00:00, 179.94s/it]


In [15]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,14990838,20000000,899,927,0.4065,0.1448,0.0864,0.000712,1.29,2.16,2012-01-22,Gw21ljO,pro_A_28 = oth460_es_ebit_fy1_d1m_l1;stock_A_2...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
1,15560946,20000000,762,803,0.5623,0.1503,0.0606,0.000534,1.28,2.48,2012-01-22,EwjOlPP,pro_A_28 = oth460_es_ebit_fy1_d1m_l1;stock_A_2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
2,16007770,20000000,465,481,0.3972,0.1546,0.0937,0.000778,1.28,2.05,2012-01-22,6wWmlWP,pro_A_28 = oth460_es_ebit_fy1_d1m_l1;stock_A_2...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
3,16257181,20000000,460,474,0.4234,0.1570,0.1071,0.000741,1.25,2.05,2012-01-22,xxLOewb,pro_A_27 = oth460_es_roe_fy1_d1m_l1;stock_A_27...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
4,14944028,20000000,888,915,0.4340,0.1443,0.0844,0.000665,1.23,2.13,2012-01-22,lVnp7JN,pro_A_27 = oth460_es_roe_fy1_d1m_l1;stock_A_27...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
5,15939527,20000000,459,476,0.4105,0.1539,0.1125,0.000750,1.22,2.00,2012-01-22,MpEMG26,pro_A_29 = oth460_es_roe_fy1_d1m_l1;stock_A_29...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
6,15252992,20000000,733,712,0.4471,0.1473,0.1227,0.000659,1.20,2.09,2012-01-22,7w57a18,pro_A_25 = oth460_1l_fbds;stock_A_25 = ts_zsco...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
7,16391414,20000000,391,380,0.4353,0.1583,0.1212,0.000727,1.20,1.99,2012-01-22,WpZ1aGx,pro_A_25 = oth460_1l_fbds;stock_A_25 = ts_zsco...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
8,15354033,20000000,755,792,0.5893,0.1483,0.0658,0.000503,1.20,2.40,2012-01-22,5wmnLRJ,pro_A_27 = oth460_es_roe_fy1_d1m_l1;stock_A_27...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
9,16097403,20000000,387,379,0.4349,0.1554,0.1124,0.000715,1.19,1.99,2012-01-22,xxLOPxw,pro_A_26 = oth460_1l_fbbs;stock_A_26 = ts_zsco...,PASS,PASS,PASS,PASS,PASS,PASS,PASS


이렇게 prettify 를 통해 시뮬레이션 한 알파들의 성능을 확인할 수 있습니다.

In [16]:
result_st1.to_csv('AB_mixing_add.csv')

In [ ]:
저장하는 것 꼭 잊지 마세요...!

In [17]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,endDate,limit,name,result,startDate,themes,value,year
0,LpvVgkv,NaN,1.58,LOW_SHARPE,PASS,NaN,NaN,2.3000,NaN
1,LpvVgkv,NaN,1.00,LOW_FITNESS,PASS,NaN,NaN,1.1500,NaN
2,LpvVgkv,NaN,0.01,LOW_TURNOVER,PASS,NaN,NaN,0.5916,NaN
3,LpvVgkv,NaN,0.70,HIGH_TURNOVER,PASS,NaN,NaN,0.5916,NaN
4,LpvVgkv,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN


In [18]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['LpvVgkv' 'PpRQw17' 'oKb83K5' '7w57a18' 'WpZ1aGx' 'lVnp7JN' 'xxLOewb'
 'Npdxb8L' 'MpEMG26' 'Gw21ljO' '6wWmlWP']
Passed alphas:['jZpz6xe', 'xxLOPxw', 'EwjOlPP', '5wmnLRJ']


In [19]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep2024'])

위 함수로 통과한 알파들에 태그를 달아줄 수 있습니다

In [9]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}
submit_result

{'ZpVprx1': False,
 'wR7RjR2': False,
 'PpLpvXq': False,
 'ZpVp0NQ': False,
 'jZkZjEk': False,
 'RpxpVv1': False,
 '1737Jak': False,
 'lVwV8Ye': False,
 'oKjKnl2': False,
 'xx7xjkn': False,
 '6w6wzxE': False,
 'z979kMK': False,
 'z979deR': False,
 'z979gzE': False,
 'jZkZ1mj': False,
 'WprpNpO': False,
 'NpEpP2p': True,
 'eOMOn0N': False,
 'RpxpeKe': False,
 'QaRaOdX': False,
 'Lpop7dn': False,
 'OpdpoKv': False,
 'p6b6x0v': False,
 'AwXwl9W': False,
 'ZpVpQMQ': False}